In [8]:
import re
import datefinder
import nltk
import spacy
import neuralcoref
from nltk.tokenize import word_tokenize

import pandas as pd
import re
from datetime import datetime
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
fake_data_path = '../epfl_server/fakes.csv'

In [9]:
fakes_df = pd.read_csv(fake_data_path, sep='|')
fakes_df.columns = ['bio']
fakes_df.head()

,bio
0,Constanzo Rossi was a Venetian Engineer. [SEP]...
1,Umberto Rizzo was a Venetian Forester. [SEP] ...
2,Adriano Rossi was a Venetian Moneylender. [SEP...
3,Adamo Bianchi was a Venetian Pardoner. [SEP] ...
4,Massimo Giordano was a Venetian Artist. [SEP] ...


In [10]:
example = fakes_df['bio'][0]

# Date Adjustment

In [11]:
borns = 0
dieds = 0
for i in range(len(fakes_df['bio'])):
    if 'born' in fakes_df['bio'][i]:
        borns += 1
    if ' die' in fakes_df['bio'][i] or 'dead' in fakes_df['bio'][i] or 'death' in fakes_df['bio'][i]:
        dieds += 1
        print(i)
print('borns: {}, dieds: {}'.format(borns/len(fakes_df['bio']), dieds/len(fakes_df['bio'])))

0
1
2
3
4
5
6
7
8
9
11
12
14
15
18
19
25
34
36
37
38
39
41
42
49
50
51
52
54
56
57
59
61
62
63
64
73
75
78
80
81
82
85
86
87
88
89
90
92
94
95
borns: 0.5625, dieds: 0.53125


In [12]:
match = re.findall(r'\d{4}', example)
sentences = nltk.tokenize.sent_tokenize(example)
dates = []
date_sentences = []
for i in match:
    for sentence in sentences:
        if i in sentence:
#             tokens = word_tokenize(sentence)
#             tokens = [word for word in tokens if word.isalpha()]
            date_sentences.append([int(i), sentence])
#             date_sentences.append(sentence)

In [13]:
df = pd.DataFrame(date_sentences,columns=['date', 'event'])
df.head()

,date,event
0,1583,"[SEP] He was born in Venice in 1583, to Gia..."
1,1809,"He joined the Navy in 1809, at the age of 21, ..."
2,1811,In 1811 he was appointed Professor of Propulsi...
3,1813,In 1813 he was appointed Director of Propulsio...
4,1814,In 1813 he was appointed Director of Propulsio...


In [14]:
sorted_events = df.sort_values('date')
sorted_events.head()

,date,event
0,1583,"[SEP] He was born in Venice in 1583, to Gia..."
1,1809,"He joined the Navy in 1809, at the age of 21, ..."
14,1810,"In 1810, he and his brother Marino completed t..."
2,1811,In 1811 he was appointed Professor of Propulsi...
25,1812,"In 1812 he was elected a member of the ""Royal ..."


In [15]:
old_sentences = []
new_sentences = []
if (list(sorted_events['date'])[-1] - sorted_events['date'][0]) > 80:
    birth_date = sorted_events['date'][1] - 20
    old_sentences.append(sorted_events['event'][0])
    new_sentences.append(sorted_events['event'][0].replace(str(sorted_events['date'][0]), str(birth_date)))

In [39]:
# check that the period between earliest and latest date
# marriage or roles 20 years after birth
# death after birth

In [17]:
if 'die' in df.iloc[-1]['event'] or 'dead' in df.iloc[-1]['event'] or 'death' in df.iloc[-1]['event']:
    if df.iloc[-1]['date'] < df.iloc[0]['date']:
        death_date = sorted_events.iloc[-1]['date'] + 5
        old_sentences.append(df.iloc[-1]['event'])
        new_sentences.append(df.iloc[-1]['event'].replace(str(df.iloc[-1]['date']), str(death_date)))

In [19]:
old_sentences, new_sentences

(['[SEP]    He was born in Venice in 1583, to Giacomo Rossi and Laura Scrini.'],
 ['[SEP]    He was born in Venice in 1789, to Giacomo Rossi and Laura Scrini.'])

In [43]:
def baseline_date_adjustment(bio):
    """
    baseline function to adjust the dates in a given biography using regex
    Args:
    bio - the biography
    Rets:
    modified_bio - the biography with the dates adjusted
    """
    match = re.findall(r'\d{4}', bio)
    sentences = nltk.tokenize.sent_tokenize(bio)
    dates = []
    date_sentences = []
    for i in match:
        for sentence in sentences:
            if i in sentence:
                date_sentences.append([int(i), sentence])
    df = pd.DataFrame(date_sentences,columns=['date', 'event'])
    sorted_events = df.sort_values('date')
    old_sentences = []
    new_sentences = []
    if (list(sorted_events['date'])[-1] - sorted_events['date'][0]) > 80:
        print((list(sorted_events['date'])[1]))
        birth_date = sorted_events['date'][1] - 20
        old_sentences.append(sorted_events['event'][0])
        new_sentences.append(sorted_events['event'][0].replace(str(sorted_events['date'][0]), str(birth_date)))
    if 'die' in df.iloc[-1]['event'] or 'dead' in df.iloc[-1]['event'] or 'death' in df.iloc[-1]['event']:
        if df.iloc[-1]['date'] < df.iloc[0]['date']:
            death_date = sorted_events.iloc[-1]['date'] + 5
            old_sentences.append(df.iloc[-1]['event'])
            new_sentences.append(df.iloc[-1]['event'].replace(str(df.iloc[-1]['date']), str(death_date)))
    modified_bio = bio[:]
    print(old_sentences, new_sentences)
    for i in range(len(old_sentences)):
        modified_bio = modified_bio.replace(old_sentences[i], new_sentences[i])
    return modified_bio

In [48]:
baseline_date_adjustment(fakes_df['bio'][5])

1587
['[SEP]       He was born in Venice in 1587, the son of Santo Lippolo and Antonio Lupo.'] ['[SEP]       He was born in Venice in 1591, the son of Santo Lippolo and Antonio Lupo.']


'Angelo Lupo was a Venetian Moneylender. [SEP]       He was born in Venice in 1591, the son of Santo Lippolo and Antonio Lupo. He attended the University of Padova and studied law and economics. In 1611, he took a position in the Court of King Louis XI, where he served as agent for the Minister of War. He returned to Venice in 1626, and was elected to the governatori of the Republic.In 1626 and 1627, Lippolo used his connections to the Medici family to influence the course of the Italian financial crisis. He served as suo jure ambassador to Venice from 1625 to 1629, and was the beneficeri of the King from 1625 to 1631. In 1626, he was the manager of the Ducal Treasury, and in 1628 he was appointed treasurer of the Republic. In the event, he was unable to participate in its management, as he and his brother Marino were still guests of the Ducal Treasury.    Lippolo\'s connections to the Medici family led him to make several policy decisions that would shape the course of Italian history

In [ ]:
# Date adjustment using coreference

In [ ]:
# EXAMPLE OF COREF USAGE

import spacy
import neuralcoref

nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)
doc1 = nlp('My sister has a dog. She loves him.')
print(doc1._.coref_clusters)

doc2 = nlp('Angela lives in Boston. She is quite happy in that city.')
for ent in doc2.ents:
    print(ent._.coref_cluster)

In [ ]:
import spacy
import neuralcoref

In [282]:
nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)
example_doc = nlp(example)
sentences_og = nltk.tokenize.sent_tokenize(example)
sentences_mod = nltk.tokenize.sent_tokenize(example_doc._.coref_resolved)
main_cluster = example_doc._.coref_clusters[0]
main_cluster_indices = [i[0].i for i in main_cluster]

In [283]:
main_cluster

Constanzo Rossi: [Constanzo Rossi, He, Giacomo Rossi, He, he, he, he, he, he, he, he, his]

In [284]:
main_cluster_indices

[0, 11, 20, 26, 55, 74, 110, 128, 143, 159, 175, 177]

In [285]:
def get_root(token):
    """
    function to get the root of a Spacy tree
    Args:
    token - a node in the tree
    Rets:
    next_ - the root of the tree where the node is
    """
    current = token
    next_ = token.head
    while (next_.text != current.text):
        current = next_
        next_ = current.head
    return next_
head = get_root(example_doc[177])

In [304]:
def all_tree_nodes(root):
    """
    function to get all nodes in a given Spacy tree
    Args:
    root - root of the tree
    Rets:
    all_nodes - a list of all the nodes in the tree
    """
    if not root.children:
        return []
    all_nodes = list(root.children)
    for child in root.children:
        all_nodes.extend(all_tree_nodes(child))
    return all_nodes


In [305]:
all_tree_nodes(get_root(example_doc[177]))

[In,
 ,,
 he,
 prototype,
 .,
 1810,
 and,
 brother,
 his,
 Marino,
 the,
 first,
 of,
 superconductor,
 the,
 ",
 ",
 ,,
 barrel,
 a,
 composed,
 of,
 condensers,
 several,
 enlarged,
 which,
 were,
 later,
 into,
 electromagnets]

In [311]:
events = []
for i in main_cluster_indices:
    subject = example_doc[i]
    head = get_root(example_doc[i])
    context = all_tree_nodes(head)
    for word in context:
        match = re.findall(r'\d{4}', word.text)
        if match:
            events.append([subject, head, word])

In [312]:
events

[[He, born, 1583],
 [Giacomo, born, 1583],
 [He, joined, 1809],
 [he, appointed, 1811],
 [he, appointed, 1813],
 [he, appointed, 1814],
 [he, appointed, 1816],
 [he, appointed, 1818],
 [he, appointed, 1822],
 [he, elected, 1824],
 [he, completed, 1810],
 [his, completed, 1810]]

In [338]:
dates = sorted([i[2] for i in events])
born_rep = -1
dead_rep = -1
born = -1
dead = -1
last_date_rep = -1
for i in events:
    if i[1].text == 'born':
        born = i[2]
    if i[1].text == 'died':
        dead = i[2]
if int(born.text) != -1:
    if int(dates[1].text) - int(born.text) < 10:
        born_rep = int(dates[1].text) - 20
if int(born.text) != -1 and int(dead.text) != -1:
    if int(dead.text) - born_rep > 100 or int(dead.text) - born_rep < 1:
        dead_rep = born_rep + 80
elif int(dates[-1].text) - int(dates[0].text) > 80:
    last_date_rep = int(dates[0].text) + 80

'1583'

In [ ]:
if born_rep != -1:
    result_doc = nlp.make_doc(example_doc[:born.i].text + f" {born_rep}" + example_doc[born.i+1:].text)
if dead_rep != -1:
    result_doc = nlp.make_doc(result_doc[:dead.i].text + f" {dead_rep}" + result_doc[dead.i+1:].text)
if last_date_rep != -1:
    result_doc = nlp.make_doc(example_doc[:dates[-1].i].text + f" {last_date_rep}" + example_doc[dates[-1].i+1:].text)

In [396]:
def nlp_sd_date_adjustment(bio):
    """
    function to adjust the dates in a given biography using NLP
    Args:
    bio - the biography
    Rets:
    the biography with the adjusted dates (string)
    """
    nlp = spacy.load('en')
    neuralcoref.add_to_pipe(nlp)
    example_doc = nlp(example)
    sentences_og = nltk.tokenize.sent_tokenize(example)
    sentences_mod = nltk.tokenize.sent_tokenize(example_doc._.coref_resolved)
    main_cluster = example_doc._.coref_clusters[0]
    main_cluster_indices = [i[0].i for i in main_cluster]
    events = []
    born_b = False
    dead_b = False
    for i in main_cluster_indices:
        subject = example_doc[i]
        head = get_root(example_doc[i])
        context = all_tree_nodes(head)
        for word in context:
            match = re.findall(r'\d{4}', word.text)
            if match:
                if not (head.text == 'born' and born_b):
                    if not(head.text == 'died' and dead_b):
                        events.append([subject, head, word])
                        if head.text == 'born':
                            born_b = True
                        if head.text == 'died':
                            dead_b = True
    dates = sorted([i[2] for i in events])
    born_rep = -1
    dead_rep = -1
    born = None
    dead = None
    last_date_rep = -1
    for i in events:
        if i[1].text == 'born':
            born = i[2]
        if i[1].text == 'died':
            dead = i[2]
    if born:
        if int(dates[1].text) - int(born.text) < 10 or int(dates[1].text) - int(born.text) > 20:
            born_rep = int(dates[1].text) - 20
    if born and dead:
        if int(dead.text) - born_rep > 100 or int(dead.text) - born_rep < 1:
            dead_rep = born_rep + 80
    elif int(dates[-1].text) - int(dates[0].text) > 80:
        if born_rep == -1:
            last_date_rep = int(dates[0].text) + 80
        else:
            last_date_rep = born_rep + 80
    if born_rep != -1:
        result_doc = nlp.make_doc(example_doc[:born.i].text + f" {born_rep}" + example_doc[born.i+1:].text)
    if dead_rep != -1:
        result_doc = nlp.make_doc(result_doc[:dead.i].text + f" {dead_rep}" + result_doc[dead.i+1:].text)
    if last_date_rep != -1:
        result_doc = nlp.make_doc(result_doc[:dates[-1].i].text + f" {last_date_rep}" + result_doc[dates[-1].i+1:].text)
    return result_doc.text

In [397]:
nlp_sd_date_adjustment(example)

'Constanzo Rossi was a Venetian Engineer. [SEP]    He was born in Venice in 1789, to Giacomo Rossi and Laura Scrini. He joined the Navy in 1809, at the age of 21, and fought in the Mediterranean Expedition in the War of the Austrian Succession. In 1811 he was appointed Professor of Propulsion and Propulsion Engineering at the Academy of Science of Berlin. In 1813 he was appointed Director of Propulsion and Propulsion Research at the Foundry of Erfurt am Sonntag in 1814, and was a member of the scientific advisory council of the Elector Institute. In 1816, he was appointed Professor of Propulsion and Propulsion Engineering at the University of Vienna. In 1818, he was appointed a Fellow of the Royal Society in London. In 1822, he was appointed Vice-Chancellor of the Royal Society in Edinburgh. In 1824, he was elected a member of the Royal Society of London.    In 1869, he and his brother Marino completed the first prototype of the "superconductor", a barrel composed of several condensers

# SCRAP FOR DATE ADJUSTMENT

In [ ]:
important_sentences_og = []
important_sentences_mod = []
main_mention = example_doc._.coref_clusters[0].main
for i in range(len(sentences_mod)):
    if str(main_mention) in sentences_mod[i] and re.findall(r'\d{4}', sentences_mod[i]):
        match = re.findall(r'\d{4}', sentences_mod[i])
        dates = match
        sentence_doc = nlp(sentences_mod[i])
                
#             print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#                     token.shape_, token.is_alpha, token.is_stop)
#         important_sentences_og.append([dates, [sentences_og[i] for j in dates], verbs])
#         important_sentences_mod.append([dates, [sentences_mod[i] for j in dates], verbs])

In [115]:
for ent in example_doc.ents:
    print(ent._.coref_cluster)

Constanzo Rossi: [Constanzo Rossi, He, Giacomo Rossi, He, he, he, he, he, he, he, he, his]
None
None
None
Constanzo Rossi: [Constanzo Rossi, He, Giacomo Rossi, He, he, he, he, he, he, he, he, his]
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
London: [London, London]
None
the Royal Society in London: [the Royal Society in London, the Royal Society, the Royal Society in Edinburgh, the Royal Society of London]
None
None
the Royal Society in London: [the Royal Society in London, the Royal Society, the Royal Society in Edinburgh, the Royal Society of London]
None
None
None
None
Friedrich Wilhelm Lechner: [Friedrich Wilhelm Lechner, He, he, He, his, he, he, his]
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
Friedrich Wilhelm Lechner: [Friedrich Wilhelm Lechner, He, he, He, his, his, he, His, his, he, his]
None
None
None
None
None
None
None
None
None
the Villa Cappello: [the Villa Cappello, the Villa Cappello]
None
None
None

In [158]:
example_doc._.coref_clusters[0][]

He

In [89]:
important_sentences_og = []
important_sentences_mod = []
main_mention = example_doc._.coref_clusters[0].main
for i in range(len(sentences_mod)):
    if str(main_mention) in sentences_mod[i] and re.findall(r'\d{4}', sentences_mod[i]):
        match = re.findall(r'\d{4}', sentences_mod[i])
        dates = match
        sentence_doc = nlp(sentences_mod[i])
                
#             print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#                     token.shape_, token.is_alpha, token.is_stop)
#         important_sentences_og.append([dates, [sentences_og[i] for j in dates], verbs])
#         important_sentences_mod.append([dates, [sentences_mod[i] for j in dates], verbs])

In [23]:
for ent in example_doc.ents:
    print(ent)
    print(ent._.coref_cluster)

Constanzo Rossi
Constanzo Rossi: [Constanzo Rossi, He, Giacomo Rossi, He, he, he, he, he, he, he, he, his]
Venetian
None
Venice
None
1583
None
Giacomo Rossi
Constanzo Rossi: [Constanzo Rossi, He, Giacomo Rossi, He, he, he, he, he, he, he, he, his]
Laura Scrini
None
Navy
None
1809
None
the age of 21
None
the Mediterranean Expedition
None
the War of the Austrian Succession
None
1811
None
the Academy of Science of Berlin
None
1813
None
Sonntag
None
1814
None
the scientific advisory council of the Elector Institute
None
1816
None
the University of Vienna
None
1818
None
a Fellow of the Royal Society
None
London
London: [London, London]
1822
None
the Royal Society
the Royal Society in London: [the Royal Society in London, the Royal Society, the Royal Society in Edinburgh, the Royal Society of London]
Edinburgh
None
1824
None
the Royal Society of London
the Royal Society in London: [the Royal Society in London, the Royal Society, the Royal Society in Edinburgh, the Royal Society of London]
18

In [197]:
import spacy
from spacy.symbols import nsubj, VERB, dobj

nlp = spacy.load("en_core_web_sm")
doc = nlp(important_sentences_og[1][1][0])
print(important_sentences_og[1][1][0])
# Finding a verb with a subject from below — good
verbs = set()
for possible_subject in doc:
#     print('subj ' + str(possible_subject))
#     print('head ' + str(possible_subject.head))
    print(possible_subject.dep, possible_subject.head.pos)
    if possible_subject.head.pos == VERB:#possible_subject.dep == nsubj and :
        verbs.add(possible_subject.head)
print(verbs)

ImportError: cannot import name 'YEAR' from 'spacy.symbols' (/Users/andrealphonse/anaconda/envs/fdhenv/lib/python3.8/site-packages/spacy/symbols.cpython-38-darwin.so)